## Importing needed packages

In [2]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

## Defining Input Data

In [7]:
# Create a dataframe for Capacity Cost
CapCost = pd.DataFrame({
    'Plant': ['Coal', 'Gas', 'Nuclear', 'Wind', 'Solar'],
    'CapCost[€/MW]': [100, 50, 200, 150, 300]
})

# Create a dataframe for Operating Cost
OpCost = pd.DataFrame({
    'Plant': ['Coal', 'Gas', 'Nuclear', 'Wind', 'Solar'],
    'OpCost[€/MWh]': [20, 80, 10, 0, 0]
})

# Create a dataframe for Storage Cost
StorCost = pd.DataFrame({
    'Plant': ['Battery', 'Pumped Hydro'],
    'StorCost[€/MWh]': [100, 50]
})

# Create a dataframe for Capacity Limit
CapLim = pd.DataFrame({
    'Plant': ['Coal', 'Gas', 'Nuclear', 'Wind', 'Solar'],
    'CapLim[MW]': [100, 100, 100, 100, 100]
})

# Create a dataframe for existing capacity
CapExi = pd.DataFrame({
    'Plant': ['Coal', 'Gas', 'Nuclear', 'Wind', 'Solar'],
    'ExCap[MW]': [100, 50, 50, 100, 100]
})

# Create a dataframe for outphased capacity
CapOut = pd.DataFrame({
    'Plant': ['Coal', 'Gas', 'Nuclear', 'Wind', 'Solar'],
    'OutCap[MW]': [0, 0, 0, 50, 0]
})
# Create a dataframe for production factor on hourly basis
ProdFac = pd.DataFrame({
    'Plant': ['Coal', 'Gas', 'Nuclear', 'Wind', 'Solar'],
    'ProdFac1': [1, 1, 1, 0.4, 0.2],
    'ProdFac2': [1, 1, 1, 0.2, 0.2],
    'ProdFac3': [1, 1, 1, 0.4, 0.2],
    'ProdFac4': [1, 1, 1, 0.5, 0.2],
    'ProdFac5': [1, 1, 1, 0.6, 0.2],
    'ProdFac6': [1, 1, 1, 0.7, 0.5],
    'ProdFac7': [1, 1, 1, 0.8, 0.2],
    'ProdFac8': [1, 1, 1, 0.3, 0.2],
    'ProdFac9': [1, 1, 1, 0.45, 0.2],
    'ProdFac10': [1, 1, 1, 0.34, 0.2],
    'ProdFac11': [1, 1, 1, 0.3, 0.2],
    'ProdFac12': [1, 1, 1, 0.3, 0.7],
    'ProdFac13': [1, 1, 1, 0.3, 0.2],
    'ProdFac14': [1, 1, 1, 0.7, 0.2],
    'ProdFac15': [1, 1, 1, 0.3, 0.2],
    'ProdFac16': [1, 1, 1, 0.3, 0.2],
    'ProdFac17': [1, 1, 1, 0.6, 0.5],
    'ProdFac18': [1, 1, 1, 0.3, 0.2],
    'ProdFac19': [1, 1, 1, 0.3, 0.2],
    'ProdFac20': [1, 1, 1, 0.2, 0.2],
    'ProdFac21': [1, 1, 1, 0.3, 0.2],
    'ProdFac22': [1, 1, 1, 0.3, 0.2],
    'ProdFac23': [1, 1, 1, 0.3, 0.2],
    'ProdFac24': [1, 1, 1, 0.3, 0.2]

})

# Create a dataframe for demand on hourly basis
Demand = pd.DataFrame({
    'Hour': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
    'Demand[MWh]': [500,600,555,343,644,343,535,223,535,634,343,535,223,535,634,343,535,223,535,634,343,535,223,535]
})

# Create a dataframe for eta charge
EtaCh = pd.DataFrame({
    'Plant': ['Battery', 'Pumped Hydro'],
    'EtaCh': [0.9, 0.8]
})

# Create a dataframe for eta discharge
EtaDis = pd.DataFrame({
    'Plant': ['Battery', 'Pumped Hydro'],
    'EtaDis': [0.9, 0.8]
})

# Create a dataframe for existing storage capacity
StorExi = pd.DataFrame({
    'Plant': ['Battery', 'Pumped Hydro'],
    'ExStor[MWh]': [40, 10]
})


## Defining Input Parameters

In [12]:
# Define number of hours as length of demand
N_Hours = len(Demand)

# Define number of generators as length of CapCost
N_Cap = len(CapCost)

# Define number of storage as length of StorCost
N_Stor = len(StorCost)

# Define epsilon
epsilon = 0.9

#Define delta
delta = 0.8

## Defining classes

In [13]:
# Class for external input data
class InputData():
    def __init__(self, CapCost, OpCost, StorCost, CapLim,CapExi,CapOut,ProdFactor,Dem,EtaCha,EtaDis,StorExi):
        self.CapCost = CapCost,
        self.OpCost = OpCost,
        self.StorCost = StorCost,
        self.CapLim = CapLim,
        self.CapExi = CapExi,
        self.CapOut = CapOut,
        self.ProdFactor = ProdFactor
        self.Dem = Dem,
        self.EtaCha = EtaCha,
        self.EtaDis = EtaDis,
        self.StorExi = StorExi

        

In [14]:
# Class for model parameters
class Parameters():
    def __init__(self, epsilon, delta, N_Hours, N_Cap, N_Stor):
        self.epsilon = epsilon,
        self.delta = delta,
        self.N_Hours = N_Hours,
        self.N_Cap = N_Cap,
        self.N_Stor = N_Stor

## Creating Data and Parameter Objects

In [16]:
ParametersObj = Parameters(epsilon, delta, N_Hours, N_Cap, N_Stor)
DataObj = InputData(CapCost, OpCost, StorCost, CapLim,CapExi,CapOut,ProdFac,Demand,EtaCh,EtaDis,StorExi)


## Creating the Model

In [21]:
# CLASS WHICH CAN HAVE ATTRIBUTES SET

class Expando(object):
    '''
        A small class which can have attributes set
    '''
    pass

In [22]:
# Defining the optimization model class

class CapacityProblem():
    def __init__(self, ParametersObj, DataObj, Model_results = 1, Guroby_results = 1):
        self.P = ParametersObj # Parameters
        self.D = DataObj # Data
        self.Model_results = Model_results
        self.Guroby_results = Guroby_results
        self.var = Expando()  # Variables
        self.con = Expando()  # Constraints
        self.res = Expando()  # Results
        self._build_model() 


    def _build_variables(self):
        # Create the variables
        self.var.CapNew = self.m.addMVar((self.P.N_Cap), lb=0)  # New Capacity for each type of generator technology
        self.var.EGen = self.m.addMVar((self.P.N_Cap, self.P.N_Hours), lb=0)  # Energy Production for each type of generator technology for each hour and scenario
        self.var.CapStor = self.m.addMVar((self.P.N_Stor), lb=0)  # New storage capacity for each type of storage technology
        self.var.SOC = self.m.addMVar((self.P.N_Stor, self.P.N_Hours), lb=0)  # State of charge for each type of storage technology for each hour and scenario  
        self.var.EChar = self.m.addMVar((self.P.N_Stor, self.P.N_Hours), lb=0)  # Energy charged for each type of storage technology for each hour and scenario
        self.var.EDis = self.m.addMVar((self.P.N_Stor, self.P.N_Hours), lb=0)  # Energy discharged for each type of storage technology for each hour and scenario


    def _build_constraints(self):
        # Limit new capacity by maximum investable capacity for each type of generator technology
        for g in range(self.P.N_Cap):
            self.con.CapLim = self.m.addConstr(self.var.CapNew[g] <= self.D.CapLim[g], name='Capacity limit')
        
        # Production limited by sum of new capacity, existing capacity and phased out capacity times the production factor
        for g in range(self.P.N_Cap):
            for h in range(self.P.N_Hours):
                #for s in range(self.P.N_Scen):
                    self.con.ProdLim = self.m.addConstr(self.var.EGen[g, h] <= (self.var.CapNew[g] + self.D.CapExi[g] + self.D.CapOut[g]) * self.D.ProdFactor[g,h], name='Production limit')

        # Energy Balance constraint - generation and discharge needs to equal demand and charging for each hour and scenario
        for h in range(self.P.N_Hours):
            #for s in range(self.P.N_Scen):
                self.con.Balance = self.m.addConstr(gp.quicksum(self.var.EGen[:, h]) + gp.quicksum(self.var.EDis[:, h]) * self.D.EtaDis == self.D.Dem[h] + gp.quicksum(self.var.EChar[:, h]), name='Energy balance')

        # Defining RES share as a percentage of total energy demand for each scenario
        # for s in range(self.P.N_Scen):
        #     self.con.RESShare = self.m.addConstr(gp.quicksum(self.var.EGen[0, :, s]) >= self.P.epsilon * gp.quicksum(self.D.Dem[:, s]), name='RES share')

        # Defining SOC as SOC from previous hour plus energy charged minus energy discharged for each hour and scenario
        for u in range(self.P.N_Stor):
            for h in range(1, self.P.N_Hours):
                #for s in range(self.P.N_Scen):
                    self.con.SOC = self.m.addConstr(self.var.SOC[u, h] == self.var.SOC[u, h-1] + self.var.EChar[u, h] * self.D.EtaCha - self.var.EDis[u, h], name='State of charge')

        # Limit SOC by maximum storage capacity for each type of storage technology
        for u in range(self.P.N_Stor):
            for h in range(self.P.N_Hours):
                #for s in range(self.P.N_Scen):
                    self.con.SOCLim = self.m.addConstr(self.var.SOC[u, h] <= self.var.CapStor[u] + self.D.StorExi[u], name='Storage capacity limit')

        # Limit charged energy by capacity minus state of charge for each hour and scenario
        for u in range(self.P.N_Stor):
            for h in range(self.P.N_Hours):
                #for s in range(self.P.N_Scen):
                    self.con.ECharLim = self.m.addConstr(self.var.EChar[u, h] * self.D.EtaCha <= self.var.CapStor[u] + self.D.StorExi[u] - self.var.SOC[u, h], name='Energy charged limit')

        # Limit discharged energy by state of charge for each hour and scenario
        for u in range(self.P.N_Stor):
            for h in range(self.P.N_Hours):
                #for s in range(self.P.N_Scen):
                    self.con.EDisLim = self.m.addConstr(self.var.EDis[u, h] <= self.var.SOC[u, h], name='Energy discharged limit')


        
    
    def _build_objective(self):
        # Objective function
        objective = gp.quicksum(self.var.CapNew[g] * self.D.CapCost[g] for g in range(self.P.N_Cap))  # Cost for new capacity for each type of generator technology
        + gp.quicksum(self.var.EGen[g,h] * self.D.OpCost[g] for g in range(self.P.N_Cap) for h in range(self.P.N_Hours)) # Operating cost for each type of generator technology for each hour and scenario
        + gp.quicksum(self.var.CapStor[u] * self.D.StorCost[u] for u in range(self.P.N_Stor)) # Cost for new storage capacity for each type of storage technology
        self.m.setObjective(objective, GRB.MINIMIZE)


    def _display_guropby_results(self):
        self.m.setParam('OutputFlag', self.Guroby_results)

    
    def _build_model(self):
        self.m = gp.Model('CapacityProblem')
        self._build_variables()
        self._build_constraints()
        self._build_objective()
        self._display_guropby_results()
        self.m.optimize()
        self._results()
        if self.Model_results == 1:
            self._extract_results()
            

    
    def _results(self):
        self.res.obj = self.m.objVal
        self.res.CapNew = self.var.CapNew.X
        self.res.EGen = self.var.EGen.X
        self.res.CapStor = self.var.CapStor.X
        self.res.SOC = self.var.SOC.X
        self.res.EChar = self.var.EChar.X
        self.res.EDis = self.var.EDis.X

    def _extract_results(self):
        # Display the objective value
        print('Objective value: ', self.m.objVal)
        # Display the new capacity for each type of generator technology
        print('New capacity for each type of generator technology: ', self.res.CapNew)
        

## Execute the model

In [23]:
CapacityProblem = CapacityProblem(ParametersObj, DataObj)

Set parameter Username


GurobiError: License expired 2025-01-29